In [1]:
import numpy as np
from grabscreen import grab_screen
import cv2
import time
from getkeys import key_check
from getgamepad import gamepad_check
import os

w = [1,0,0,0,0,0,0,0,0]
s = [0,1,0,0,0,0,0,0,0]
a = [0,0,1,0,0,0,0,0,0]
d = [0,0,0,1,0,0,0,0,0]
wa = [0,0,0,0,1,0,0,0,0]
wd = [0,0,0,0,0,1,0,0,0]
sa = [0,0,0,0,0,0,1,0,0]
sd = [0,0,0,0,0,0,0,1,0]
nk = [0,0,0,0,0,0,0,0,1]


In [2]:
starting_value = 1

while True:
    file_name = 'training_data-{}.npy'.format(starting_value)

    if os.path.isfile(file_name):
        print('File exists, moving along',starting_value)
        starting_value += 1
    else:
        print('File does not exist, starting fresh!',starting_value)
        
        break

File exists, moving along 1
File does not exist, starting fresh! 2


In [3]:
def keys_to_output(keys):
    '''
    Convert keys to a ...multi-hot... array
     0  1  2  3  4   5   6   7    8
    [W, S, A, D, WA, WD, SA, SD, NOKEY] boolean values.
    '''
    output = [0,0,0,0,0,0,0,0,0]

    if 'W' in keys and 'A' in keys:
        output = wa
    elif 'W' in keys and 'D' in keys:
        output = wd
    elif 'S' in keys and 'A' in keys:
        output = sa
    elif 'S' in keys and 'D' in keys:
        output = sd
    elif 'W' in keys:
        output = w
    elif 'S' in keys:
        output = s
    elif 'A' in keys:
        output = a
    elif 'D' in keys:
        output = d
    else:
        output = nk
    return output

def gamepad_keys_to_output(lista):
    '''
    Convert list of boolean to values 0 or 1
    ['LT','RT','Lx','Ly','Rx','Ry','UP','DOWN','LEFT','RIGHT','START','SELECT','L3','R3','LB','RB','A','B','X','Y']
    example
    [0, 0, 1792, -2099, 1272, -1997, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
    '''
    output = [int(elem) for elem in lista]
    return output

In [4]:
# Done by Frannecklp

import cv2
import numpy as np
import win32gui, win32ui, win32con, win32api

def grab_screen(region=None):

    hwin = win32gui.GetDesktopWindow()

    if region:
            left,top,x2,y2 = region
            width = x2 - left + 1
            height = y2 - top + 1
    else:
        width = win32api.GetSystemMetrics(win32con.SM_CXVIRTUALSCREEN)
        height = win32api.GetSystemMetrics(win32con.SM_CYVIRTUALSCREEN)
        left = win32api.GetSystemMetrics(win32con.SM_XVIRTUALSCREEN)
        top = win32api.GetSystemMetrics(win32con.SM_YVIRTUALSCREEN)

    hwindc = win32gui.GetWindowDC(hwin)
    srcdc = win32ui.CreateDCFromHandle(hwindc)
    memdc = srcdc.CreateCompatibleDC()
    bmp = win32ui.CreateBitmap()
    bmp.CreateCompatibleBitmap(srcdc, width, height)
    memdc.SelectObject(bmp)
    memdc.BitBlt((0, 0), (width, height), srcdc, (left, top), win32con.SRCCOPY)
    
    signedIntsArray = bmp.GetBitmapBits(True)
    img = np.fromstring(signedIntsArray, dtype='uint8')
    img.shape = (height,width,4)

    srcdc.DeleteDC()
    memdc.DeleteDC()
    win32gui.ReleaseDC(hwin, hwindc)
    win32gui.DeleteObject(bmp.GetHandle())

    return cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)

In [5]:

import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from numpy import load

In [6]:
def process_horizontal(img):
    # Median filter is used to suppress image noise. The edge of the signal can be protected from blurring while noise is filtered out.
    img = cv.medianBlur(img,9)
    # Simple threshold
    ret, frame = cv.threshold(img,100,255,cv.THRESH_BINARY) #night/day
    # edge detection
    processed_img = cv2.Sobel(frame,cv2.CV_64F,0 ,1, ksize=5) #Horizontal Gradient
    return processed_img


In [7]:
def process_imagen(img):
    # Median filter is used to suppress image noise. The edge of the signal can be protected from blurring while noise is filtered out.
    img = cv.medianBlur(img,9)
    # Simple threshold
    ret, frame = cv.threshold(img,100,255,cv.THRESH_BINARY) #night/day
    # edge detection
    processed_img = cv2.Canny(frame, threshold1 = 200, threshold2=300)
   # y= cv2.Sobel(frame,cv2.CV_64F,0 ,1, ksize=5) #Horizontal Gradient
    return processed_img


In [8]:
def process_red(img):
    # Median filter is used to suppress image noise. The edge of the signal can be protected from blurring while noise is filtered out.
    img = cv.medianBlur(img,5)
    # Simple threshold
    #ret,th1 = cv.threshold(img,121,255,cv.THRESH_BINARY) #day
    #ret,th1 = cv.threshold(img,61,255,cv.THRESH_BINARY) #night
    ret,th1 = cv.threshold(img,100,255,cv.THRESH_BINARY) #night/day
    frame=th1
    y= cv2.Sobel(frame,cv2.CV_64F,0 ,1, ksize=5) #Horizontal Gradient
    #edge= cv2.Canny(frame, 100, 200)#Edge Detection
    src =y
    #print(src.shape)
    # extract red channel
    red_channel = src[:,:,2]
    # create empty image with same shape as that of src image
    red_img = np.zeros(src.shape)
    #assign the red channel of src to empty image
    red_img[:,:,2] = red_channel
    return red_img


In [9]:
def parmeterv(x):
    y= 0.8660735375167304*x + 23.147783639083652
    return y

In [10]:
def parameter(img):
    #img = cv2.imread('way-night.png')
    #img = cv2.imread('way-day.png')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    oned=img.ravel()
    #y, x, _ = plt.hist(oned)
    histog=np.histogram(oned)
    y, x = histog[0],histog[1]
    # Get the maximum element from a Numpy array
    y_max = np.amax(y)
    # Get the indices of maximum element in numpy array
    result = np.where(y == np.amax(y))
    position_max=result[0].item()
    x_max=x[position_max]
    #print(x_max)
    intensity = parmeterv(x_max)
    return  intensity



In [11]:
#img = cv2.imread('way-day.png')
#img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#parameter(img)

In [12]:
def process_black_contour(img):
    image = img

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #thresh = cv2.threshold(gray,90, 255,cv2.THRESH_BINARY_INV)[1] # only for night
    #thresh = cv2.threshold(gray,200, 255,cv2.THRESH_BINARY_INV)[1] # only for day  
    intensity=parameter(image)
    thresh = cv2.threshold(gray,intensity, 255,cv2.THRESH_BINARY_INV)[1] # any time or weather
    cnts, h = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    label = '1'
    count = 0

    # Get inner list of hierarchy
    for layer in zip(cnts, h[0]):
        contour = layer[0]
        hierarchy = layer[1]

        # If we find new contour (not inner) reset label
        if hierarchy[1] >= 0:
            label = '1'
        # Ensure that we only have outer contour
        if count % 2 == 0:
            cv2.drawContours(image, [contour], -1, (36, 255, 12), 2)
            x,y,w,h = cv2.boundingRect(contour)
            cv2.putText(image, label, (x +50,y+ 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (36,255,12), 3)
            label = str(int(label) * -1)

        count += 1
    return thresh

In [13]:
from skimage import io
from sklearn.cluster import KMeans, MiniBatchKMeans
import numpy as np
import time
import sys
def _extract_features(img):
    h, w, ch = img.shape
    features = img.reshape((h*w, ch))
    return features
def _clusterize(colors, n_clusters):
    t = time.time()
    kmeans = MiniBatchKMeans(n_clusters=n_clusters,
                             random_state=0,
                             batch_size=2**12).fit(colors)
    #print('Time to clusterize: {} s'.format(time.time() - t))

    return kmeans.labels_, kmeans.cluster_centers_
def _reduce_colors(output_shape, labels, labels_colors):
    out = np.zeros((labels.size, 3))
  
    for label, color in enumerate(labels_colors):
        out[labels == label] = color

    out = out.reshape(output_shape).astype(np.uint8)
    return out
def color_quantization(img, n_colors=3):
    colors = _extract_features(img)
    labels, labels_colors = _clusterize(colors, n_colors)
    out = _reduce_colors(img.shape, labels, labels_colors)
    return  out

In [18]:
def main(file_name, starting_value):
    file_name = file_name
    starting_value = starting_value
    training_data = []
    for i in list(range(4))[::-1]:
        print(i+1)
        time.sleep(1)

    last_time = time.time()
    paused = False
    print('STARTING!!!')
    while(True):
        
        if not paused:
            screen = grab_screen(region=(0,40,1920,1120))

            # 800x600 windowed mode
            #screen = grab_screen(region=(0,40,800,640))
            last_time = time.time()
            # resize to something a bit more acceptable for a CNN
            screen = cv2.resize(screen, (480,270))
            #screen = cv2.resize(screen, (160,120))
            # run a color convert:
            screen = cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)
            #screen=process_red(screen)
            #screen=process_imagen(screen)
            #screen=process_horizontal(screen)
            #screen=color_quantization(screen)
            screen = process_black_contour(screen)
            keys = key_check()
            #print(keys)
            #print(type(keys))
            output = keys_to_output(keys)
            #print(output)
            #print(type(output))
            gamepad_keys = gamepad_check()
            #print(gamepad_keys)
            #print(type(gamepad_keys))
            output_gamepad = gamepad_keys_to_output(gamepad_keys)
            #print(output_gamepad)
            #print(type(output_gamepad))
            training_data.append([screen,output,output_gamepad])
            #print('loop took {} seconds'.format(time.time()-last_time))
            last_time = time.time()
            cv2.imshow('window',cv2.resize(screen,(640,360)))
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break

            if len(training_data) % 100 == 0:
                print(len(training_data))
                
                if len(training_data) == 500:
                    #np.save(file_name,training_data)
                    print('SAVED')
                    #training_data = []
                    #starting_value += 1
                    #file_name = 'training_data-{}.npy'.format(starting_value)
               
        keys = key_check()
        if 'T' in keys:
            if paused:
                paused = False
                print('unpaused!')
                time.sleep(1)
            else:
                print('Pausing!')
                paused = True
                time.sleep(1)

In [19]:
main(file_name, starting_value)

4
3
2
1
STARTING!!!


C:\Anaconda3\envs\pygta5\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


100
200
300
400
500
SAVED
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
